In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import cPickle as pickle

plt.style.use('ggplot')

In [2]:
cd ..

/Users/odatakuma/cmu/FleetAI


In [3]:
data_path = 'analytics/data/taxi_trips/taxi_tripdata_2016-05.csv'

In [4]:
cols = ['dayofweek', 'hour', 'minute', 'pickup_latitude', 'pickup_longitude',
        'dropoff_latitude', 'dropoff_longitude', 'great_circle_distance', 'trip_distance', 'trip_time']
df = pd.read_csv(data_path, usecols=cols, dtype={key : np.float32 for key in cols})
df.head()

,dayofweek,dropoff_latitude,dropoff_longitude,hour,minute,pickup_latitude,pickup_longitude,trip_distance,trip_time,great_circle_distance
0,6,40.730099,-73.983986,0,0,40.768040,-73.985901,3.60,17.516666,4221
1,6,40.744633,-73.980995,0,0,40.741573,-73.993073,1.09,7.016667,1072
2,6,40.733002,-74.002258,0,0,40.684601,-73.991943,4.21,19.783333,5451
3,6,40.765469,-73.975700,0,0,40.744751,-73.991577,1.68,7.516667,2663
4,6,40.737564,-73.997498,0,0,40.740192,-74.005280,0.56,6.650000,717


In [5]:
df['hour'] = df.hour + df.minute/60
df = df.drop('minute', axis=1)
df['trip_distance'] = df.trip_distance * 1.60934
df['great_circle_distance'] = df.great_circle_distance / 1000
df = df[df.trip_distance > df.great_circle_distance]
df = df[df.trip_distance < df.great_circle_distance*5]

In [6]:
df.shape

(12275709, 9)

In [7]:
from sklearn.cross_validation import train_test_split

X = df[['dayofweek', 'hour', 'pickup_latitude', 'pickup_longitude',
        'dropoff_latitude', 'dropoff_longitude', 'trip_distance']].values
y = df['trip_time'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print X.shape

/Users/odatakuma/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(12275709, 7)


In [8]:
%%time
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(learning_rate=1.0, n_estimators=20, subsample=0.2, max_depth=4)
gbr.fit(X_train, y_train)
rmse_train = np.sqrt(((y_train - gbr.predict(X_train))**2).mean())
rmse_test = np.sqrt(((y_test - gbr.predict(X_test))**2).mean())
print "RMSE train/val: %.1f / %.1f" % (rmse_train, rmse_test)

RMSE train/val: 5.3 / 5.3
CPU times: user 6min 40s, sys: 9.05 s, total: 6min 49s
Wall time: 6min 56s


In [13]:
%%time
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators=10, min_samples_split=2e-5, n_jobs=-1)
forest.fit(X_train, y_train)
rmse_train = np.sqrt(((y_train - forest.predict(X_train))**2).mean())
rmse_test = np.sqrt(((y_test - forest.predict(X_test))**2).mean())
print "RMSE train/val: %.1f / %.1f" % (rmse_train, rmse_test)

RMSE train/val: 4.3 / 4.6
CPU times: user 27min 49s, sys: 6.93 s, total: 27min 56s
Wall time: 8min 29s


In [17]:
from keras.models import Sequential
from keras.layers.core import Dense

def MLP(size, act='relu'):
    model = Sequential()
    model.add(Dense(size, input_dim=7, activation=act))
    model.add(Dense(size, activation=act))
    model.add(Dense(size, activation=act))
    model.add(Dense(1, init='normal'))
    model.compile(loss='mean_squared_error',  optimizer='adam')
    return model

In [18]:
%%time
mlp = MLP(64)
mlp.fit(X_train, y_train, nb_epoch=10, batch_size=1000, verbose=0, validation_split=0.0)
y_pred_train = mlp.predict(X_train, batch_size=10000, verbose=0)[:, 0]
y_pred_test = mlp.predict(X_test, batch_size=10000, verbose=0)[:, 0]
rmse_train = np.sqrt(((y_train - y_pred_train)**2).mean())
rmse_test = np.sqrt(((y_test - y_pred_test)**2).mean())
print "RMSE train/val: %.1f / %.1f" % (rmse_train, rmse_test)                         

RMSE train/val: 5.7 / 5.7
CPU times: user 19min 26s, sys: 2min 24s, total: 21min 50s
Wall time: 8min 43s


In [19]:
%%time
forest = RandomForestRegressor(n_estimators=10, min_samples_split=5e-5, n_jobs=-1)
forest.fit(X_train, y_train)
rmse_train = np.sqrt(((y_train - forest.predict(X_train))**2).mean())
rmse_test = np.sqrt(((y_test - forest.predict(X_test))**2).mean())
print "RMSE train/val: %.1f / %.1f" % (rmse_train, rmse_test)

RMSE train/val: 4.6 / 4.7
CPU times: user 26min 56s, sys: 12 s, total: 27min 8s
Wall time: 8min 46s


In [20]:
path = 'engine/model/pickle/triptime_predictor.pkl'
with open(path, 'wb') as f:
    pickle.dump(forest, f)